In [1]:
import pickle
# NetworkX is a Python package for the creation, manipulation,
# and study of the structure, dynamics, and functions of complex networks.
import networkx as nx
# NumPy est une extension du langage de programmation Python, destinée à manipuler des matrices ou
# tableaux multidimensionnels ainsi que des fonctions mathématiques opérant sur ces tableaux. 
import numpy as np # Statistic

In [2]:
# characters = Dictionnaire des chapitres et des nom
with open('GOT-characters-raw.pickle','rb') as fp:
    characters = pickle.load(fp)
len(characters.keys())

340

In [3]:
# set() : Est un module qui créer une liste d'éléments uniques
# entities = collection
entities = set()
for chapter in characters.values(): # .values() = Liste les valeurs de chaque keys
    for entity in chapter:
        name = entity[0] # entities = chaque valeur d'une keys
        entities.add(name)
print(len(entities))

5522


In [4]:
# Cré un graph
GOT = nx.Graph()
print(nx.info(GOT))

Name: 
Type: Graph
Number of nodes: 0
Number of edges: 0



In [10]:
chapter = characters['1–1-Bran']
# sort = tri en ordre croissant
# lambda a:a[1] = pour chaque key, prend l'élément 1 de la clé puis trier les ensembles avec sort
chapter.sort(key=lambda a:a[1],reverse=False)
# print(chapter)
# [('Bran', 159, 163), ('Robb', 463, 467), ('Mance Rayder', 514, 526), ('Old Nan', 629, 636),

In [13]:
def get_edges_from_chapter(chapter):
    potential_edges = []
    for i in range(0,len(chapter)): # prend le 1er de la liste
        name1,start1,end1 = chapter[i]
        for j in range(i+1,len(chapter)):
            name2,start2,end2 = chapter[j]
            if name2 != name1:
                potential_edges.append((name1,name2,{'dist':start2-end1}))
    # print(len(potential_edges))
    # print(potential_edges)
    # [('Tyrion Lannister', 'Starks', {'dist': 35}), ('Tyrion Lannister', 'Tyrion', {'dist': 468})
    
    edges = {}
    for name1,name2,value in potential_edges:
        if (name1,name2) not in edges.keys():
            if (name2,name1) not in edges.keys():
                edges[(name1,name2)] = {'values':[value['dist']]}
            else:
                edges[(name2,name1)]['values'].append(value['dist'])
        else:
            edges[(name1,name2)]['values'].append(value['dist'])
    
    # print(edges)
    # {('Gared', 'Ser Waymar Royce'): {'values': [102, 932, 2608, 8611, 9229, 18382, 20037,
    
    for k,v in edges.items():
        # 25: 25%
        # percentile: écarts types
        # average / median / percentile
        # edges[k]['weight']=np.percentile(v['values'],25)
        edges[k]['weight']=len([a for a in v['values'] if a <1000])
        # print(k,edges[k]['weight'])

    ready_for_nx = []
    for k,v in edges.items():
        name1 = k[0]
        name2 = k[1]
        weight = v['weight']
        if weight > 0:
            ready_for_nx.append((name1,name2,{'weight':weight}))
    return ready_for_nx

In [14]:
for chapter in characters.values():
    GOT.add_edges_from(get_edges_from_chapter(chapter))

In [82]:
print(nx.info(GOT))

Name: 
Type: Graph
Number of nodes: 5520
Number of edges: 141887
Average degree:  51.4083


In [83]:
nx.write_gexf(GOT,'GOT-version-1.gexf')